In [1]:
# Read file
try:
    with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
        text = f.read()
except FileNotFoundError:
    # Get tinyshakespeare dataset to train the model on
    !wget -O tinyshakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
    
    with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
        text = f.read()

In [2]:
# Inspect
print(text[:100])

print("-----")
print("Length of dataset:{}".format(len(text)))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
-----
Length of dataset:1115394


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print("{} characters".format(vocab_size))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65 characters


### Tokenization

In [22]:
# We define a simple mapping between characters and integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda i: "".join([itos[n] for n in i])

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [30]:
# We can compare the encoding to more complex methods, such as OpenAI's tiktoken
import tiktoken
enc = tiktoken.get_encoding("gpt2")

print("Vocabulary size: {}".format(enc.n_vocab))
enc.encode("Hello world!")

Vocabulary size: 50257


[15496, 995, 0]

In [42]:
# We will stick to our simple mapping and tokenize the inut dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [49]:
# And do a simple split of the dataset for training and validation
ratio = .9
train_data = data[:int(ratio*len(data))]
val_data = data[int(ratio*len(data)):]

### Data Loader

We will start by building our data loader. This will need to consider two dimensions:
*  **Block size**: Context length provided to generate any given prediction.
*  **Batch size**: Number of independent sequences that are going to be processed in parallel.

In [59]:
block_size = 8
batch_size = 4

In [81]:
torch.manual_seed(1337)
def get_batch(split):
    #Generate a batch of inputs x and targets y
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch("train")
print("Inputs:")
print(xb)
print("Outputs:")
print(yb)

Inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Outputs:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


Note that this simple tensor actually contains a total of 32 independent training examples, as each batch has a total of 8 training examples with varying context size.

The transformer is going to be trained simultaneously on both for two reasons:
1. Computational reasons: Parallel computing of multiple batches optimizes GPU usage.
1. Inference reasons: This way, we also train the transformer to infer targets from different given context lengths.

In [84]:
for batch in range(batch_size):
    for ex in range(block_size):
        context = xb[batch, :ex+1]
        target = yb[batch, ex]
        print(f"Given context {context.tolist()}, the following token is {target}")

Given context [24], the following token is 43
Given context [24, 43], the following token is 58
Given context [24, 43, 58], the following token is 5
Given context [24, 43, 58, 5], the following token is 57
Given context [24, 43, 58, 5, 57], the following token is 1
Given context [24, 43, 58, 5, 57, 1], the following token is 46
Given context [24, 43, 58, 5, 57, 1, 46], the following token is 43
Given context [24, 43, 58, 5, 57, 1, 46, 43], the following token is 39
Given context [44], the following token is 53
Given context [44, 53], the following token is 56
Given context [44, 53, 56], the following token is 1
Given context [44, 53, 56, 1], the following token is 58
Given context [44, 53, 56, 1, 58], the following token is 46
Given context [44, 53, 56, 1, 58, 46], the following token is 39
Given context [44, 53, 56, 1, 58, 46, 39], the following token is 58
Given context [44, 53, 56, 1, 58, 46, 39, 58], the following token is 1
Given context [52], the following token is 58
Given conte